# 🎯 MOUAADNET-ULTRA Detection Training

**Author:** MOUAAD IDOUFKIR

---

In [ ]:
#@title 1️⃣ Setup
!nvidia-smi
!pip install -q torch torchvision tqdm pycocotools opencv-python onnx

In [ ]:
#@title 2️⃣ Config
import os

WORK = '/teamspace/studios/this_studio' if os.path.exists('/teamspace') else '/content'
REPO = f'{WORK}/MouaadNet-Ultra'
DATA = f'{WORK}/coco'
CKPT = f'{WORK}/checkpoints'

os.makedirs(DATA, exist_ok=True)
os.makedirs(CKPT, exist_ok=True)
print(f'Work: {WORK}\nData: {DATA}\nCheckpoints: {CKPT}')

In [ ]:
#@title 3️⃣ Clone Repo
if not os.path.exists(REPO):
    !git clone https://github.com/mouuuuaad/MouaadNet-Ultra.git {REPO}
else:
    !cd {REPO} && git pull
os.chdir(REPO)
print(f'✅ {os.getcwd()}')

In [ ]:
#@title 4️⃣ Download COCO (Full + Verify)
import os
import subprocess

def count_files(path):
    if not os.path.exists(path):
        return 0
    return len([f for f in os.listdir(path) if f.endswith('.jpg')])

# Expected counts
EXPECTED_TRAIN = 118287
EXPECTED_VAL = 5000

train_count = count_files(f'{DATA}/train2017')
val_count = count_files(f'{DATA}/val2017')
anno_exists = os.path.exists(f'{DATA}/annotations/instances_train2017.json')

print(f'Train images: {train_count}/{EXPECTED_TRAIN}')
print(f'Val images: {val_count}/{EXPECTED_VAL}')
print(f'Annotations: {"✅" if anno_exists else "❌"}')

# Re-download if incomplete
if train_count < EXPECTED_TRAIN * 0.99:
    print('\n📥 Downloading train2017 (~18GB)...')
    !rm -rf {DATA}/train2017 {DATA}/train2017.zip
    !wget -q --show-progress http://images.cocodataset.org/zips/train2017.zip -O {DATA}/train2017.zip
    !cd {DATA} && unzip -q train2017.zip && rm train2017.zip
    print(f'✅ Train: {count_files(f"{DATA}/train2017")} images')

if val_count < EXPECTED_VAL * 0.99:
    print('\n📥 Downloading val2017 (~1GB)...')
    !rm -rf {DATA}/val2017 {DATA}/val2017.zip
    !wget -q --show-progress http://images.cocodataset.org/zips/val2017.zip -O {DATA}/val2017.zip
    !cd {DATA} && unzip -q val2017.zip && rm val2017.zip
    print(f'✅ Val: {count_files(f"{DATA}/val2017")} images')

if not anno_exists:
    print('\n📥 Downloading annotations...')
    !rm -rf {DATA}/annotations {DATA}/annotations.zip
    !wget -q --show-progress http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O {DATA}/annotations.zip
    !cd {DATA} && unzip -q annotations.zip && rm annotations.zip

print('\n✅ COCO Dataset Ready!')

In [ ]:
#@title 5️⃣ Train
EPOCHS = 50
BATCH_SIZE = 16
LR = 0.001

!python training/train_detection.py \
    --data {DATA} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --lr {LR} \
    --save-dir {CKPT} \
    --export

In [ ]:
#@title 6️⃣ Results
import torch
import matplotlib.pyplot as plt

ckpt = torch.load(f'{CKPT}/best.pt', map_location='cpu')
history = ckpt.get('history', {})

if history:
    plt.figure(figsize=(10, 4))
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Val')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.title(f'Best: {ckpt["best_loss"]:.4f}')
    plt.savefig(f'{CKPT}/curves.png')
    plt.show()

print(f'\nBest: {ckpt["best_loss"]:.4f}')

In [ ]:
#@title 7️⃣ Download
print('📦 Models:')
for f in os.listdir(CKPT):
    if f.endswith(('.pt', '.onnx')):
        print(f'   {f} ({os.path.getsize(f"{CKPT}/{f}")/1e6:.1f}MB)')

try:
    from google.colab import files
    files.download(f'{CKPT}/best.pt')
    files.download(f'{CKPT}/detection.onnx')
except:
    print(f'\n📁 Files at: {CKPT}')